In [139]:
import tsplib95 
import networkx as nx
import numpy as np
np.random.seed(42)

problem = tsplib95.load("ca4663.tsp")
nodes = problem.node_coords

In [58]:
problem.as_name_dict().keys()

dict_keys(['name', 'comment', 'type', 'dimension', 'edge_weight_type', 'node_coords'])

In [831]:
class TSPSolver:
    def solve(self,nodes):
        self.nodes = nodes
        initial_solution = self.initial_solution()
        best_solution = self.local_search(initial_solution)

    def initial_solution(self):
        """path is the node_codes in order, cost is the from previous to current node."""
        n = len(self.nodes)
        # path = np.random.permutation(np.arange(1,n+1,1)) 

        path = list(self.nodes.keys())
        cost = self.cost(path)
        solution = {"path":path} | cost
        return solution

    def local_search(self, current_solution):
        self.simulated_annealing(current_solution)

    def simulated_annealing(self, current_solution):
        current_path = current_solution["path"]
        current_eval = current_solution["total_cost"]
        initial_eval = current_eval
        cooling_rate = 100
        non_improov = 0
        max_non_improov = 10

        temperature = 6000
        n = len(current_path)

        while non_improov < max_non_improov and temperature > 0:
            i,j = np.random.choice(range(1,n-1),size=2,replace=False)
            bounds = (i,j) if i<j else (j,i)
            swap_cost = self.swap_cost(current_path,bounds)

            if swap_cost < 0:
                current_path = self.swap_n_reverse(current_path, (i,j))
                current_eval += swap_cost
            else:
                non_improov += 1
                accepting_prob = np.e**(swap_cost/temperature)
                if np.random.uniform() < accepting_prob:
                    current_path = self.swap_n_reverse(current_path, (i,j))
                    current_eval += swap_cost
                
            temperature -= cooling_rate

    
        print(self.check_eval(current_eval,current_path, swap_cost))
        print(f"initial cost:{initial_eval}; current cost: {current_eval}; {current_eval/initial_eval}% \n ({i,j})")

    def node_distance(self,a,b):
        distance = self.distance(self.nodes[a], self.nodes[b])
        return distance

    def distance(self, a, b):
        #todo: check performance
        return np.sqrt(
        (a[0] - b[0])**2 + (a[1] - b[1])**2
        )

        
    def swap_n_reverse(self, path, bounds):
        new_path = np.copy(path)
        i,j = bounds
        new_path[i:j] = new_path[i:j][::-1]

        return new_path


    def check_eval(self, current_eval,current_path, swap_cost):
        calculated_cost = self.cost(current_path)["total_cost"]
        diff = calculated_cost - current_eval + 0.0000001
        print(f"\n swap/diff: {swap_cost/diff} \n eval: {current_eval}; calculated: {calculated_cost}; diff: {diff} diff% {current_eval/calculated_cost}")

        return ((calculated_cost - current_eval)**2)**0.5 < 100


    def swap_cost(self, path, bounds):
        i,j = bounds
        n1,n2 = path[i-1], path[i]
        n3,n4 = path[j-1], path[j]

        d = self.node_distance
        cost = (d(n1,n2) - d(n1,n3)) + (d(n3,n4) - d(n2,n4))

        # test_path = np.copy(path[i-1:j+1])
        # calc_cost_before = self.cost(test_path)["total_cost"]
        # test_path[1:-1] = test_path[1:-1][::-1]
        # calc_cost_after = self.cost(test_path)["total_cost"]
        # calc_cost = calc_cost_before - calc_cost_after

        # curr_cost = d(path[i-1],path[i]) + d(path[j],path[j+1])
        # new_cost = d(path[i-1],path[j]) + d(path[i],path[j+1])
        # swap_cost = new_cost - curr_cost

        # print(f"swap_cost: {cost}; cost_calc: {calc_cost}; %{cost/calc_cost}")

        return cost
            
  
    def cost(self, path):
        n = len(path)
        cost = np.array([self.distance(self.nodes[path[i]],self.nodes[path[i+1]]) for i in range(n-1)])
        total_cost = sum(cost)
        # cost = np.zeros(n)
        # list(map(self.distance, path))
        # cost = [ self.di
        # stance(a,b) for a,b in path]
        return {"cost":cost, "total_cost":total_cost}

In [837]:
# node = (1,0)
# edges = problem.get_edges()
# node = next(edges)
# while node[0] == 1:
#     node = next(edges)
#     print(node)
#All the nodes are connected
# d_l
# aa = [1,2,3,4,5]
# aa[2:4]
# np.array(aa)[2:4]

array = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
array_s = [ 1,  2,  6,  5,  4,  3,  7,  8,  9, 10]
 
solver.swap_cost(array,(2,6)) == solver.cost(array)["total_cost"] - solver.cost(array_s)["total_cost"]


swap_cost: -51.64984133771401; cost_calc: -51.64984133771395; %1.000000000000001


True

In [832]:
solver = TSPSolver()
# solver.swap_n_reverse(a)
# solver.solve(dic)
# solver.solve(nodes)
# d_l = list(dic.keys())
# solver.swap_n_reverse(d_l,(1,8))
# solver.nodes = dic
# solver.swap_cost(list(dic.keys()),[2,3])
solver.solve(nodes)


swap_cost: -57189.7228293698; cost_calc: -57189.722829375416; %0.9999999999999017
swap_cost: -7911.043838678057; cost_calc: -7911.04383867234; %1.0000000000007228
swap_cost: -18067.95265807181; cost_calc: -18067.952658071794; %1.0000000000000009
swap_cost: -19120.908388323864; cost_calc: -19120.90838831663; %1.0000000000003781
swap_cost: -8242.62204916605; cost_calc: -8242.622049175203; %0.9999999999988896
swap_cost: -41930.76632470095; cost_calc: -41930.76632471755; %0.999999999999604
swap_cost: -82561.9080875453; cost_calc: -82561.90808754507; %1.0000000000000027
swap_cost: -802.6072268013304; cost_calc: -802.6072267889977; %1.000000000015366
swap_cost: -2384.5510133333487; cost_calc: -2384.5510133728385; %0.9999999999834394
swap_cost: -34978.32958803804; cost_calc: -34978.32958805561; %0.9999999999994976
swap_cost: -74.35777341911307; cost_calc: -74.35777341760695; %1.0000000000202551
swap_cost: -364.3779610679085; cost_calc: -364.3779610544443; %1.000000000036951
swap_cost: -596.74

In [434]:
p1 = (1,1)
p2 = (2,2)
p3 = (3,3)
p4 = (4,4)
p5 = (5,5)
e1 = (p1,p2)
e2 = (p2,p3)
e3 = (p3, p4)
e4 = (p4, p5)
d = solver.distance
original = solver.distance(p1, p2) + solver.distance(p2, p3) + solver.distance(p3, p4) + solver.distance(p4, p5)
changed = solver.distance(p1, p4) + solver.distance(p4, p3) + solver.distance(p3, p2) + solver.distance(p2, p5)
diff = (d(p1,p2) - d(p1,p4)) + (d(p4,p5)-d(p2,p5))

print(original, changed, diff, original - changed, original - diff)

5.656854249492381 11.31370849898476 -5.65685424949238 -5.656854249492379 11.31370849898476


In [36]:
d = {"a":1, "b":2}
d.pop("a")
d

{'b': 2}

In [715]:
solver = TSPSolver()
solver.solve(nodes)

old: 3947.233677818809 cost: 3947.2336778188087;

 swap/diff: -0.762032254764082 
 eval: 47851477.10356175; calculated: 47846297.22613108; diff: -5179.8774305699635 diff% 1.0001082607794327
False
initial cost:47847529.86988393; current cost: 47851477.10356175; 1.0000824960805408% 
 ((3117, 3238))
